In [240]:
import math
import json
import requests
import itertools
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import re
import ast

In [241]:
movies_df = pd.read_csv('movies_metadata.csv')
movies_df

/var/folders/ny/vyccgwl94n39q3y435x80g540000gn/T/ipykernel_22171/3749947666.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [242]:
# drop rows with no genre
movies_df = movies_df[movies_df['genres'] != '[]']
movies_df = movies_df.reset_index(drop=True)
movies_df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43019,False,NaN,0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,222848,tt0112613,en,Caged Heat 3000,It's the year 3000 AD. The world's most danger...,...,1995-01-01,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Caged Heat 3000,False,3.5,1.0
43020,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",...,1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,False,5.7,26.0
43021,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
43022,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0


In [243]:
# 250 movies
movies = movies_df[:250]
movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,False,"{'id': 217704, 'name': 'The Vampire Chronicles...",60000000,"[{'id': 27, 'name': 'Horror'}, {'id': 10749, '...",NaN,628,tt0110148,en,Interview with the Vampire,"A vampire relates his epic life story of love,...",...,1994-11-11,223664608.0,123.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Drink from me and live forever,Interview with the Vampire,False,7.2,1558.0
246,False,NaN,14000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,87729,tt0113463,en,Jefferson in Paris,"His wife having recently died, Thomas Jefferso...",...,1995-03-31,2474000.0,139.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A powerful man torn between his love for one w...,Jefferson in Paris,False,5.2,10.0
247,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",NaN,18713,tt0110189,en,The Jerky Boys,When two unemployed telephone pranksters decid...,...,1995-02-03,0.0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Jerky Boys,False,4.8,10.0
248,False,NaN,60000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",NaN,6280,tt0110216,en,Junior,"As part of a fertility research project, a mal...",...,1994-11-22,37000000.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Nothing is inconceivable,Junior,False,4.7,381.0


In [244]:
movies.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [245]:
movies["genres"][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [246]:
# convert string representation of a list of dictionaries to a list of dictionaries
movies["genres"] = movies["genres"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

/var/folders/ny/vyccgwl94n39q3y435x80g540000gn/T/ipykernel_22171/282178170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres"] = movies["genres"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [247]:
movies["genres"][0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [248]:
def get_pushshift_data(query, **kwargs):
    """
    Gets data from the pushshift api.
 
    args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    """
 
    base_url = f"https://api.pushshift.io/reddit/search/comment/?q={query}&subreddit=movies&score=2"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.json()

In [254]:
data_list = []
result = {'data': []}

for index, row in movies.iterrows():
    movie_title = row["title"]
    movie_genres = row["genres"]

    genre_names = [genre['name'] for genre in movie_genres]
    genre_string = ', '.join(genre_names)
    
    response = get_pushshift_data(     
                       query=movie_title,                 
                       after='365d',          
                       size='100',
                       aggs=aggs)
        
    for r in response['data']:
        r['movie_name'] = movie_title
        r['genre'] = genre_string
    data_list.append(response)
    
for data in data_list:
    result['data'].extend(data['data'])

In [255]:
rows = [{'movie_name': row['movie_name'],
         'genre': row['genre'],
         'author': row['author'],
         'score': row['score'],
         'body': row['body'],
        'permalink': row['permalink'],
         'link_id': row['link_id'],
         'utc_datetime_str': row['utc_datetime_str'],
        }for row in result['data']]

In [256]:
reddit = pd.DataFrame(rows)

In [257]:
reddit

,movie_name,genre,author,score,body,permalink,link_id,utc_datetime_str
0,Toy Story,"Animation, Comedy, Family",JeanMorel,1,Wikipedia has a whole article about this. Sett...,/r/movies/comments/10wyl7v/what_are_the_big_ci...,t3_10wyl7v,2023-02-08 15:46:31
1,Toy Story,"Animation, Comedy, Family",thegimboid,1,Or Tom Hanks and Tim Allen in Toy Story,/r/movies/comments/10wawc4/is_there_a_better_l...,t3_10wawc4,2023-02-08 05:15:47
2,Toy Story,"Animation, Comedy, Family",JPumpkinhead1991,1,It had no effect on me. I saw it once and neve...,/r/movies/comments/10wj59e/goonies_the_fratell...,t3_10wj59e,2023-02-08 02:53:04
3,Toy Story,"Animation, Comedy, Family",IllustriousDebt6248,1,I myself did not need a Toy Story 4.,/r/movies/comments/160v0a/pixar_vs_dreamworks_...,t3_160v0a,2023-02-08 02:38:45
4,Toy Story,"Animation, Comedy, Family",GabbiStowned,1,"I did this, but listing my favorite movie for ...",/r/movies/comments/10wd94y/what_is_in_your_opi...,t3_10wd94y,2023-02-07 23:48:35
...,...,...,...,...,...,...,...,...
16947,Just Cause,"Action, Drama, Mystery, Thriller",JamesJFresh,1,"*[Just a warning, my response turned into more...",/r/movies/comments/10mad06/official_discussion...,t3_10mad06,2023-02-03 08:49:50
16948,Just Cause,"Action, Drama, Mystery, Thriller",mechaiineramen,1,&gt;Avatar is more about the radicalization of...,/r/movies/comments/zugf0n/movies_shower_though...,t3_zugf0n,2023-02-03 06:45:12
16949,Just Cause,"Action, Drama, Mystery, Thriller",LiteraryBoner,1,"Damn, I kind of loved this? And it's not that ...",/r/movies/comments/10s98qm/official_discussion...,t3_10s98qm,2023-02-03 03:09:17
16950,Just Cause,"Action, Drama, Mystery, Thriller",chlamydial_lips,1,I agree with it bearing many off the hallmarks...,/r/movies/comments/10rrbrs/nine_years_ago_toda...,t3_10rrbrs,2023-02-03 00:00:56


In [258]:
reddit.to_csv('reddit_pushshift.csv')

In [261]:
# drop rows that contains 'bot' in author column

reddit_no_bots = reddit.loc[~reddit['author'].str.contains('bot', case=False), :]

In [262]:
reddit_no_bots

,movie_name,genre,author,score,body,permalink,link_id,utc_datetime_str
0,Toy Story,"Animation, Comedy, Family",JeanMorel,1,Wikipedia has a whole article about this. Sett...,/r/movies/comments/10wyl7v/what_are_the_big_ci...,t3_10wyl7v,2023-02-08 15:46:31
1,Toy Story,"Animation, Comedy, Family",thegimboid,1,Or Tom Hanks and Tim Allen in Toy Story,/r/movies/comments/10wawc4/is_there_a_better_l...,t3_10wawc4,2023-02-08 05:15:47
2,Toy Story,"Animation, Comedy, Family",JPumpkinhead1991,1,It had no effect on me. I saw it once and neve...,/r/movies/comments/10wj59e/goonies_the_fratell...,t3_10wj59e,2023-02-08 02:53:04
3,Toy Story,"Animation, Comedy, Family",IllustriousDebt6248,1,I myself did not need a Toy Story 4.,/r/movies/comments/160v0a/pixar_vs_dreamworks_...,t3_160v0a,2023-02-08 02:38:45
4,Toy Story,"Animation, Comedy, Family",GabbiStowned,1,"I did this, but listing my favorite movie for ...",/r/movies/comments/10wd94y/what_is_in_your_opi...,t3_10wd94y,2023-02-07 23:48:35
...,...,...,...,...,...,...,...,...
16947,Just Cause,"Action, Drama, Mystery, Thriller",JamesJFresh,1,"*[Just a warning, my response turned into more...",/r/movies/comments/10mad06/official_discussion...,t3_10mad06,2023-02-03 08:49:50
16948,Just Cause,"Action, Drama, Mystery, Thriller",mechaiineramen,1,&gt;Avatar is more about the radicalization of...,/r/movies/comments/zugf0n/movies_shower_though...,t3_zugf0n,2023-02-03 06:45:12
16949,Just Cause,"Action, Drama, Mystery, Thriller",LiteraryBoner,1,"Damn, I kind of loved this? And it's not that ...",/r/movies/comments/10s98qm/official_discussion...,t3_10s98qm,2023-02-03 03:09:17
16950,Just Cause,"Action, Drama, Mystery, Thriller",chlamydial_lips,1,I agree with it bearing many off the hallmarks...,/r/movies/comments/10rrbrs/nine_years_ago_toda...,t3_10rrbrs,2023-02-03 00:00:56


In [263]:
reddit_no_bots.to_csv('reddit_pushshift_remove_bots.csv')